In [1]:
import numpy as np
import pandas as pd
import random
from queue import PriorityQueue
from sklearn.cluster import KMeans

# 데이터 파일 로드
def load_data(filename):
    data = pd.read_csv(filename, header=None)
    cities = data.to_numpy() #numpy배열이 대용량 다차원 데이터를 효율적인 연산으로 다룸
    return cities 

# 유전 알고리즘을 수행
class GeneticAlgorithm:
    # 객체 초기화
    def __init__(self, cities, population_size, mutation_rate, generations):
        self.cities = cities
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        
    # 하나의 개체(=하나의 경로/하나의 군집의 전체 경로)를 생성
    def create_individual(self):
        individual = np.arange(0, len(self.cities))#도시의 갯수만큼 배열을 생성해놓고
        np.random.shuffle(individual[1:])#첫번째를 제외하고 셔플을 돌린다
        return individual#셔플된 배열을 반환한다

    # 전체 개체 집단을 생성
    def create_population(self):
        return np.array([self.create_individual() for _ in range(self.population_size)])
    
    # 개체의 적합도 계산
    def fitness(self, individual):
        total_distance = 0
        for i in range(len(individual) - 1):#거리는 저넻 도시갯수 -1 개이고 첫지점과 마지막 지점을 잇는거는 for문 밖에 설정한다
            total_distance += np.linalg.norm(self.cities[individual[i]] - self.cities[individual[i + 1]]) # 셔플된 도시들의 인덱스를 이용해서 거리를 계산후 total_distance 에 추가한다
        total_distance += np.linalg.norm(self.cities[individual[-1]] - self.cities[0])  # 마지막 지점에서 첫 출발지점까지 이어 추가한다
        return -total_distance #거리가 짧을 수록 좋기 때문에 -를 붙여서 fitness를 정한다

    # 개체 집단을 적합도 순으로 정렬
    def rank_population(self, population):
        fitness_results = np.array([self.fitness(individual) for individual in population])
        return list(np.argsort(fitness_results))

    #  부모 개체 선택
    def selection(self, population, ranked_population):
        selected_indices = []
        for i in range(len(ranked_population)):
            if random.random() < (i + 1) / len(ranked_population):
                selected_indices.append(ranked_population[i])
        return population[selected_indices]
    
    # 개체의 BFS 서브트리를 생성
    def bfs_subtree(self, individual, start_node):
        queue = [start_node]
        subtree = {start_node}  # 시작 노드는 subtree에 포함
        while queue:
            node = queue.pop(0)
            for neighbor in individual:
                if neighbor not in subtree and neighbor not in queue:
                    queue.append(neighbor)
                    subtree.add(neighbor)
        return subtree

    
    # 교차 - BFS 활용
    def crossover(self, parent1, parent2):
        child = np.zeros(len(parent1), dtype=int)
        child[0] = parent1[0]  # 첫 번째 노드는 항상 고정

        parent1_subtree = self.bfs_subtree(parent1, child[0])
        parent2_subtree = self.bfs_subtree(parent2, child[0])

        for i in range(1, len(parent1)):
            if parent1[i] in parent1_subtree:
                child[i] = parent1[i]
            elif parent1[i] in parent2_subtree:
                child[i] = parent2[parent1.tolist().index(parent1[i])]

        return child

    # 변이 - 더블 브리지
    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            size = len(individual)
            pos1 = random.randint(1, size//4)
            pos2 = random.randint(pos1+1, size//2)
            pos3 = random.randint(pos2+1, 3*size//4)
            pos4 = random.randint(pos3+1, size-1)
            
            p1 = individual[:pos1]
            p2 = individual[pos2:pos3]
            p3 = individual[pos4:]
            p4 = individual[pos1:pos2]
            p5 = individual[pos3:pos4]
            
            individual = np.concatenate((p1, p2, p3, p4, p5))
        return individual

    # 실행 - 최적의 개체와 거리를 반환
    def run(self):
        population = self.create_population()

        for generation in range(self.generations):
            print(f"Generation {generation + 1}")

            ranked_population = self.rank_population(population)
            selected_population = self.selection(population, ranked_population)

            children = []
            while len(children) < len(population):
                parent1 = random.choice(selected_population)
                parent2 = random.choice(selected_population)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                children.append(child)
            population = np.array(children)

        best_individual_index = self.rank_population(population)[0]
        best_individual = population[best_individual_index]
        best_individual_distance = -self.fitness(best_individual)

        return best_individual.reshape(-1), best_individual_distance

# A* 휴리스틱을 사용하여 군집 간 최단 경로를 탐색
class AStarHeuristic:
    # 객체 초기화
    def __init__(self, cities, paths):
        self.cities = cities
        self.paths = paths

    # 두 도시 간의 유클리드 거리 계산
    def euclidean_distance(self, city1, city2):
        return np.linalg.norm(self.cities[city1] - self.cities[city2])

    # 경로 재구성
    def reconstruct_path(self, came_from, start, goal):
        path = [goal]
        current = goal # 제일 끝 경로부터 시작 경로까지 거꾸로 추적
        while current != start:
            current = came_from[current]
            path.append(current)
        path.reverse() # 실제 순서대로 구성
        return path

    # 실행 - 시작 도시에서 목표 도시까지의 최단 경로와 거리를 반환
    def run(self, goal):
        start = 0
        open_set = PriorityQueue()
        open_set.put((0, start))

        came_from = {}
        g_score = {city: float('inf') for city in range(len(self.cities))}
        g_score[start] = 0
        f_score = {city: float('inf') for city in range(len(self.cities))}
        f_score[start] = self.euclidean_distance(start, goal)

        open_set_hash = {start}

        while not open_set.empty():
            current = open_set.get()[1]
            open_set_hash.remove(current)

            if current == goal:
                return self.reconstruct_path(came_from, start, goal), g_score[goal]

            for neighbor in self.paths[current]:
                tentative_g_score = g_score[current] + self.euclidean_distance(current, neighbor)

                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = g_score[neighbor] + self.euclidean_distance(neighbor, goal)

                    if neighbor not in open_set_hash:
                        open_set.put((f_score[neighbor], neighbor))
                        open_set_hash.add(neighbor)
        return None, float('inf')

# k-means 군집화 수행
class KMeansCluster:
    def __init__(self, k, cities):
        self.k = k
        self.cities = cities

    def cluster(self):
        kmeans = KMeans(n_clusters=self.k, random_state=42).fit(self.cities)#KMeans의 fit 메서드를 통해서 학습시킨다
        return kmeans.labels_#학습 이후 도시들이 해당되는 label을 반환한다
    
class TSP:
    # 객체 초기화, 데이터 로드
    def __init__(self, data_filename):
        self.data_filename = data_filename
        self.cities = load_data(self.data_filename)

    # TSP 문제 해결
    def solve(self):
        k = 5 # 군집은 5개로 지정
        k_means = KMeansCluster(k, self.cities)
        cluster_labels = k_means.cluster()

        # 각 군집에 대해 유전 알고리즘을 실행, 결과 저장
        cluster_results = []
        for i in range(k):
            cluster_cities = self.cities[cluster_labels == i]
            ga = GeneticAlgorithm(cluster_cities, population_size=50, mutation_rate=0.1, generations=1000)
            best_individual, best_individual_distance = ga.run()
            cluster_results.append((best_individual, best_individual_distance))

        # 나눠진 군집은 A* 알고리즘을 사용하여 merge
        merged_path, merged_distance = self.merge_paths(cluster_results)

        return merged_path, merged_distance
    
    # TSP 문제의 최종 경로를 합치는 과정
    def merge_paths(self, paths):
        paths = sorted(paths, key=lambda x: x[1])  # 경로를 거리가 짧은 순서로 정렬
        start = paths.pop(0)  # start에서 가장 짧은 경로를 찾아 이어 붙임

        while paths:
            closest_path, closest_distance = None, float('inf')
            closest_index = -1
            for i, (path, distance) in enumerate(paths):
                # 가장 가까운 경로를 찾는 알고리즘은 A star 휴리스틱 사용
                a_star = AStarHeuristic(self.cities, {start[0][-1]: [path[0]]})
                new_path, new_distance = a_star.run(path[0])

                if new_distance < closest_distance:
                    closest_path, closest_distance = path, new_distance
                    closest_index = i

            # start와 가장 가까운 군집 경로를 결합
            paths.pop(closest_index)
            start = (np.concatenate([start[0], closest_path]), start[1] + closest_distance)

        return start

    # 최단 경로와 총 거리를 출력
    def display_result(self, path, distance):
        print("최단 경로: ", path)
        print("총 거리: ", distance)

# 데이터 로드, TSP solve() 호출
import time

tsp = TSP("2023_AI_TSP.csv")

start_time = time.time()
final_path, final_distance = tsp.solve()
end_time = time.time()

# 결과 출력
tsp.display_result(final_path, final_distance)

# 최단 경로 저장
with open('example_solution.csv', 'w+') as f:
    for city in final_path:
        f.write(f"{city}\n")

# 실행 시간 출력
print(f"Execution time: {end_time - start_time} seconds")


Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30
Generation 31
Generation 32
Generation 33
Generation 34
Generation 35
Generation 36
Generation 37
Generation 38
Generation 39
Generation 40
Generation 41
Generation 42
Generation 43
Generation 44
Generation 45
Generation 46
Generation 47
Generation 48
Generation 49
Generation 50
Generation 51
Generation 52
Generation 53
Generation 54
Generation 55
Generation 56
Generation 57
Generation 58
Generation 59
Generation 60
Generation 61
Generation 62
Generation 63
Generation 64
Generation 65
Generation 66
Generation 67
Generation 68
Generation 69
Generation 70
Generation 71
Generation 72
G

KeyboardInterrupt: 